# [Module 3.2] Personalize 솔류션 버전 평가 지표 확인하기

이 노트북은 아래와 같은 작업을 합니다.

* 솔류션 버전으로 부터 성능 지표 구하기
* 성능 지표 분석

---
노트북의 실행 소요 시간은 약 1분 입니다.

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time


다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [3]:
%store -r

생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

## 솔루션 버전 평가를 위한 학습 데이터 분리 방법

이번 파트에서는 Amazon Personalize에서 기본으로 제공하는 솔루션에 대한 평가 지표를 확인해 봅니다. 
Amazon Personalize에서는 평가 지표를 생성하기 위해 약 랜덤으로 10% 사용자의 interaction data를 테스트 용으로 활용합니다. 

아래 이미지는 Amazon Personalize가 데이터를 분리하는 방법을 보여줍니다. 사용자가 10 명이고 각각 10 개의 상호 작용이있는 경우 (여기에서 원은 Interaction data를 나타냄) 타임 스탬프를 기준으로 가장 오래된 것부터 최신 것까지 나열된 것입니다. Amazon Personalize는 사용자의 90 % (파란색 원)의 모든 Interaction 데이터를 사용하여 솔루션 버전을 훈련시키고 나머지 10 %는 평가를 위해 사용합니다. 나머지 10 %의 각 사용자에 대해 Interaction data (녹색 원)의 90 %가 훈련 된 모델의 입력값으로 사용됩니다. 데이터의 나머지 10 % (주황색 원)는 모델에서 생성 된 추천 결과물과 비교되고 평가 지표를 계산하는 데 사용됩니다.



![personalize metrics](static/imgs/personalize_metrics.png)



#### 솔류션 버전의 성능 지표 구하기



In [4]:
metrics=[]

def build_metric_matrix(solution,response):
    metrics.append([solution,
                response['metrics']['coverage'],
                response['metrics']['mean_reciprocal_rank_at_25'],
                response['metrics']['normalized_discounted_cumulative_gain_at_5'],
                response['metrics']['normalized_discounted_cumulative_gain_at_10'],
                response['metrics']['normalized_discounted_cumulative_gain_at_25'],
                response['metrics']['precision_at_5'],
                response['metrics']['precision_at_10'],
                response['metrics']['precision_at_25']])

#### Metrics: Popularity

아래는 생성한 7개의 솔류션 버전에 대해서 성능 지표를 얻습니다.

In [5]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = popularity_solution_version_arn 
)

print(json.dumps(get_solution_metrics_response, indent=2))
build_metric_matrix('popularity',get_solution_metrics_response)


{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-popularity-08866/b9eddfc3",
  "metrics": {
    "coverage": 0.0136,
    "mean_reciprocal_rank_at_25": 0.0449,
    "normalized_discounted_cumulative_gain_at_10": 0.056,
    "normalized_discounted_cumulative_gain_at_25": 0.0914,
    "normalized_discounted_cumulative_gain_at_5": 0.0346,
    "precision_at_10": 0.0158,
    "precision_at_25": 0.0142,
    "precision_at_5": 0.0155
  },
  "ResponseMetadata": {
    "RequestId": "fa2dc05a-4a0b-41bc-9f43-390a9900ee56",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 26 Feb 2023 02:39:59 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "414",
      "connection": "keep-alive",
      "x-amzn-requestid": "fa2dc05a-4a0b-41bc-9f43-390a9900ee56"
    },
    "RetryAttempts": 0
  }
}


#### Metrics: User-Personalization

In [6]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = user_personalization_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

build_metric_matrix('user_personalization',get_solution_metrics_response)

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-user-personalization-08866/02d2bad5",
  "metrics": {
    "coverage": 0.2838,
    "normalized_discounted_cumulative_gain_at_5": 0.2083,
    "normalized_discounted_cumulative_gain_at_25": 0.3549,
    "precision_at_25": 0.066,
    "normalized_discounted_cumulative_gain_at_10": 0.2748,
    "precision_at_10": 0.0944,
    "precision_at_5": 0.1104,
    "mean_reciprocal_rank_at_25": 0.2629
  },
  "ResponseMetadata": {
    "RequestId": "a443cee8-9d65-4981-9b83-66b3568bc6a5",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 26 Feb 2023 02:40:00 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "424",
      "connection": "keep-alive",
      "x-amzn-requestid": "a443cee8-9d65-4981-9b83-66b3568bc6a5"
    },
    "RetryAttempts": 0
  }
}


#### Metrics: Similar Items

In [7]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = sims_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))
build_metric_matrix('Similar-Items',get_solution_metrics_response)

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-similar-items-08866/8ef71484",
  "metrics": {
    "coverage": 0.2552,
    "mean_reciprocal_rank_at_25": 0.1873,
    "normalized_discounted_cumulative_gain_at_10": 0.1976,
    "normalized_discounted_cumulative_gain_at_25": 0.263,
    "normalized_discounted_cumulative_gain_at_5": 0.1509,
    "precision_at_10": 0.0588,
    "precision_at_25": 0.0425,
    "precision_at_5": 0.069
  },
  "ResponseMetadata": {
    "RequestId": "82db0814-77cd-430c-8936-ffe69a3115da",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 26 Feb 2023 02:40:01 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "416",
      "connection": "keep-alive",
      "x-amzn-requestid": "82db0814-77cd-430c-8936-ffe69a3115da"
    },
    "RetryAttempts": 0
  }
}


#### Metrics: Ranking

In [8]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = ranking_solution_version_arn 
)

print(json.dumps(get_solution_metrics_response, indent=2))
build_metric_matrix('ranking',get_solution_metrics_response)

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-ranking-08866/4fd50926",
  "metrics": {
    "coverage": 0.2865,
    "mean_reciprocal_rank_at_25": 0.2652,
    "normalized_discounted_cumulative_gain_at_10": 0.2625,
    "normalized_discounted_cumulative_gain_at_25": 0.3474,
    "normalized_discounted_cumulative_gain_at_5": 0.2027,
    "precision_at_10": 0.0907,
    "precision_at_25": 0.0656,
    "precision_at_5": 0.1078
  },
  "ResponseMetadata": {
    "RequestId": "cde4a6df-6b30-4e91-b5f3-f20cb3ccc7d3",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 26 Feb 2023 02:40:03 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "412",
      "connection": "keep-alive",
      "x-amzn-requestid": "cde4a6df-6b30-4e91-b5f3-f20cb3ccc7d3"
    },
    "RetryAttempts": 0
  }
}


## Summary Metrics

#### 1. 솔류션 평가 지표 정의
[솔류션 평가 지표 정의](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html)
는 개발자 문서의 링크 참조 바랍니다. 또한 이 링크 [솔류션 평가 정의 예제](http://francescopochetti.com/recommend-expedia-hotels-with-amazon-personalize-the-magic-of-hierarchical-rnns/) 의 페이지 맨 아래 쪽을 보시면 조금 더 직관적인 그림을 보실 수 있습니다.
 <br>
또한 reciprocal_rank_at_5, normalized_discounted_cumulative_gain_at_5,precision_at_5 의 예제는 아래와 같습니다. 
* Exmaple
    * 5 개의 추천리스트를 제공했고, 이 중에 2번째와 5번째가 실제 데이타와 일치 했다고 하면, 쉽게 이렇게 [0,1,0,0,1] 표시 할 수 있습니다.
        * reciprocal_rank
            * 1/2 (0.5) # 가장 빠른 순서의 하나만을 선택 합니다
        * normalized_discounted_cumulative_gain_at_5
            * (1/log(1+2) + 1/log(1+5)) / (1/log(1+1) + 1/log(1+2)) = 0.6241
        * precision_at_5
            * 2/5 (0.4)

#### 2. 솔류션 버전의 성능 지표 분석

레서피의 종류는 크게 세가지 입니다. 각각에 대해 확인을 해보겠습니다.
#### 1. USER_PERSONALIZATION Recipes
- 5가지 (popularity,user_personalization,hrnn,hrnn_meta,hrnn_coldstart) 있습니다.
- popularity 는 베이스라인의 레서피로서 샤용을 주로 합니다. 모든 지표에서 가장 낮은 수치를 보입니다.
- hrnn, hrnn-meta, user_personalization 이 전반적으로 가장 높은 성능을 보여 줍니다. user_personalization은 default로 exploration_weight=0.3 입니다. coldstart item 이 거의 없는 상태에서도 높은 성능을 보여 주고 있습니다.학습 데이터의 인터렉션 데이터에 콜드 스타트 아이템이 전형 없기에 성능은 감소합니다. 이 경우에는 exloration_weight = 0을 하게 되면 콜드 스타트의 아이템을 배제하고 추천 합니다.<br>
이 링크 [exploration_weight](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-new-item-USER_PERSONALIZATION.html)에서 "Item Exploration Configuration Hyperparameters" 이 부분을 참고 하세요.<br>
- hrnn_coldstart 는coldstart item 이 거의 없는 상태이기에 성능이 낮게 나오는 것이 정상 으로 보입니다.

#### 2. RELATED_ITEMS Recipes
- sims 가 부류에 속하는 레서피로 커버리지가 높게 나왔습니다.

#### 3. PERSONALIZED_RANKING Recipes
- ranking 이 여기에 속합니다. 


<div>
    <img src="static/imgs/Fig.3.2.metric_summary2.png" width="800">
</div>

---
참고:
*추천시에 시간에 대한 입력값이 들어가기에 추천 결과가 아래 이미지 (저자 실행)와 다를 수 있습니다. 여러 번 테스트 시에 매번 조금씩 다르게 나옴을 확인 하였습니다.<br>
레서비의 정의는 개발자 문서 참고 하세요 --> [레서피 정의](https://docs.aws.amazon.com/personalize/latest/dg/working-with-predefined-recipes.html)


In [9]:
recipe_metrics=pd.DataFrame(metrics,columns=['recipe','coverage','mrr@25','ndcg@5','ndcg@10','ndcg@25','p@5','p@10','p@25'])

recipe_metrics

,recipe,coverage,mrr@25,ndcg@5,ndcg@10,ndcg@25,p@5,p@10,p@25
0,popularity,0.0136,0.0449,0.0346,0.0560,0.0914,0.0155,0.0158,0.0142
1,user_personalization,0.2838,0.2629,0.2083,0.2748,0.3549,0.1104,0.0944,0.0660
2,Similar-Items,0.2552,0.1873,0.1509,0.1976,0.2630,0.0690,0.0588,0.0425
3,ranking,0.2865,0.2652,0.2027,0.2625,0.3474,0.1078,0.0907,0.0656


In [10]:
%store user_personalization_solution_version_arn
%store user_personalization_solution_arn

%store sims_solution_version_arn
%store sims_solution_arn


Stored 'user_personalization_solution_version_arn' (str)
Stored 'user_personalization_solution_arn' (str)
Stored 'sims_solution_version_arn' (str)
Stored 'sims_solution_arn' (str)


In [11]:
%store recipe_metrics

Stored 'recipe_metrics' (DataFrame)
